# Memory

LLMs are stateless, they don't have any memory to keep conversations, we have to append the previous conversation to their inputs.
LangChain provides numerous ways to create a memory for LLMs including:

- ConversationBufferMemory: A simple memory that store the conversation.
- ConversationBufferWindowMemory: A memory that store the conversation in a window of fixed size. 
- ConversationTokenBufferMemory" A memory that store the conversation, limiting the number of tokens.
- ConversationSummaryMemory: A memory that store the conversation, summarizing it. 

Additionally, there are other interesing memories:
- Vector data memory: stores text in a vector database. We can then retrieve the most relevant blocks of text.
- Entity memories: remember specific entities. E.g. Remember facts about a friend.

We can also combine memories, e.g., Conversation memory + Entity memory to recall individuals.

## Imports

In [1]:
from dotenv import load_dotenv
import os

import openai

_ = load_dotenv("config.env")
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
model_name = "gpt-3.5-turbo-0301"

## Memory

`ConversationBufferMemory` objects are a conveninant way to create a memory for the LLM. 

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [13]:
# Initalize the model
llm = ChatOpenAI(temperature=0.0, model=model_name)

# Create a memory to store the conversation
memory = ConversationBufferMemory()

# Create a conversation chain
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [14]:
conversation.predict(input="Hi, my name is Becaye")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Becaye
AI:

> Finished chain.


"Hello Becaye, it's nice to meet you. My name is AI. How can I assist you today?"

In [15]:
conversation.predict(input="I struggle with this question: 1 + 1 = ?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Becaye
AI: Hello Becaye, it's nice to meet you. My name is AI. How can I assist you today?
Human: I struggle with this question: 1 + 1 = ?
AI:

> Finished chain.


"The answer to 1 + 1 is 2. It's a basic arithmetic problem that involves adding two numbers together. Is there anything else I can help you with?"

In [16]:
conversation.predict(input="Basic problem? You mean I am dumb?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Becaye
AI: Hello Becaye, it's nice to meet you. My name is AI. How can I assist you today?
Human: I struggle with this question: 1 + 1 = ?
AI: The answer to 1 + 1 is 2. It's a basic arithmetic problem that involves adding two numbers together. Is there anything else I can help you with?
Human: Basic problem? You mean I am dumb?
AI:

> Finished chain.


"No, not at all! Everyone has different strengths and weaknesses, and it's perfectly normal to struggle with certain types of problems. Plus, it's always good to ask questions and seek help when you need it."

In [17]:
conversation.predict(input="What is my name, again?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Becaye
AI: Hello Becaye, it's nice to meet you. My name is AI. How can I assist you today?
Human: I struggle with this question: 1 + 1 = ?
AI: The answer to 1 + 1 is 2. It's a basic arithmetic problem that involves adding two numbers together. Is there anything else I can help you with?
Human: Basic problem? You mean I am dumb?
AI: No, not at all! Everyone has different strengths and weaknesses, and it's perfectly normal to struggle with certain types of problems. Plus, it's always good to ask questions and seek help when you need it.
Human: What is my name, again?
AI:

> Finished chain.


'Your name is Becaye.'

In [19]:
print(memory.buffer)

Human: Hi, my name is Becaye
AI: Hello Becaye, it's nice to meet you. My name is AI. How can I assist you today?
Human: I struggle with this question: 1 + 1 = ?
AI: The answer to 1 + 1 is 2. It's a basic arithmetic problem that involves adding two numbers together. Is there anything else I can help you with?
Human: Basic problem? You mean I am dumb?
AI: No, not at all! Everyone has different strengths and weaknesses, and it's perfectly normal to struggle with certain types of problems. Plus, it's always good to ask questions and seek help when you need it.
Human: What is my name, again?
AI: Your name is Becaye.


In [27]:
memory.load_memory_variables(inputs={})

{'history': "Human: Hi, my name is Becaye\nAI: Hello Becaye, it's nice to meet you. My name is AI. How can I assist you today?\nHuman: I struggle with this question: 1 + 1 = ?\nAI: The answer to 1 + 1 is 2. It's a basic arithmetic problem that involves adding two numbers together. Is there anything else I can help you with?\nHuman: Basic problem? You mean I am dumb?\nAI: No, not at all! Everyone has different strengths and weaknesses, and it's perfectly normal to struggle with certain types of problems. Plus, it's always good to ask questions and seek help when you need it.\nHuman: What is my name, again?\nAI: Your name is Becaye."}

### Manually managing memory

We can manually add inputs and outputs to the memory.

In [28]:
memory = ConversationBufferMemory()
memory.save_context({"input": "Hi"}, {"output": "What's up?"})
print(memory.buffer)

Human: Hi
AI: What's up?


In [31]:
memory.save_context(
    {"input": "Not much, just hanging"}, 
    {"output": "Cool"}
)

print(memory.buffer)

Human: Hi
AI: What's up?
Human: Not much, just hanging
AI: Cool
Human: Not much, just hanging
AI: Cool


In [33]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up?\nHuman: Not much, just hanging\nAI: Cool\nHuman: Not much, just hanging\nAI: Cool"}

## Managing Context Window

The `ConversationBufferWindowMemory` allows us to control the lenght of the message history that is fed to the LLM.

In [34]:
from langchain.memory import ConversationBufferWindowMemory

In [36]:
# k=1 means we just keep the one entrance from the chatbot 
# and one from the user (the last message in our case)
memory = ConversationBufferWindowMemory(k=1)

# Store the conversation
memory.save_context({"input": "Hi"}, {"output": "What's up?"})
memory.save_context({"input": "Not much, just hanging"}, {"output": "Cool"})

# The memory kept only the last message from the user and the chatbot
memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

**Create a conversation chain with limited memory**

Let's define a memory of 1 and see how it affects the ConversationChain. 

In [38]:
llm = ChatOpenAI(temperature=0.0, model=model_name)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

conversation.predict(input="Hi, my name is Becaye")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Becaye
AI:

> Finished chain.


"Hello Becaye, it's nice to meet you. My name is AI. How can I assist you today?"

In [39]:
conversation.predict(input="I struggle with this question: 1 + 1 = ?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Becaye
AI: Hello Becaye, it's nice to meet you. My name is AI. How can I assist you today?
Human: I struggle with this question: 1 + 1 = ?
AI:

> Finished chain.


"The answer to 1 + 1 is 2. It's a basic arithmetic problem that involves adding two numbers together. Is there anything else I can help you with?"

In [40]:
conversation.predict(input="What is my name, again?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: I struggle with this question: 1 + 1 = ?
AI: The answer to 1 + 1 is 2. It's a basic arithmetic problem that involves adding two numbers together. Is there anything else I can help you with?
Human: What is my name, again?
AI:

> Finished chain.


"I'm sorry, but I don't have access to that information. Could you please remind me of your name?"

Of course, the model does not remember my name. It only remembers the last messages from the human and the bot.

## Managing Token Length

The `ConversationTokenBufferMemory` object allows us to limit the number of tokens we send to the LLM.
Unlike the `ConversationBufferWindowMemory`, the `ConversationTokenBufferMemory` does not limit the number of messages, but the number of tokens.

In [44]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI

llm = ChatOpenAI(temperature=0.0, model=model_name)

In [46]:
# We specify the OPenAI LLM because different LLMs have ways of counting tokens
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=50)

memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

In [47]:
memory.load_memory_variables({})

{'history': 'AI: Amazing!\nHuman: Backpropagation is what?\nAI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

## Summarize Conversation Memory

We can also use the `ConversationSummaryMemory` to summarize the conversation history.

In [48]:
from langchain.memory import ConversationSummaryBufferMemory

In [51]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

# Create a summary memory
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)

# Store the conversation
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

In [55]:
# Load the stored conversation
memory.load_memory_variables({})

{'history': "System: The human and AI engage in small talk before discussing the day's schedule. The AI informs the human of a morning meeting with the product team, time to work on the LangChain project, and a lunch meeting with a customer interested in the latest AI developments."}

We can see that the conversation have been summarized.

In [57]:
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI engage in small talk before discussing the day's schedule. The AI informs the human of a morning meeting with the product team, time to work on the LangChain project, and a lunch meeting with a customer interested in the latest AI developments.
Human: What would be a good demo to show?
AI:

> Finished chain.


"Based on the customer's interest in AI developments, I would suggest showcasing our latest natural language processing capabilities. We could demonstrate how our AI can accurately understand and respond to complex language queries, and even provide personalized recommendations based on the user's preferences. Additionally, we could highlight our AI's ability to learn and adapt over time, making it a valuable tool for businesses looking to improve their customer experience."

In [61]:
history = memory.load_memory_variables({})["history"]

[print(h) for h in history.split("\n")]

System: The human and AI engage in small talk before discussing the day's schedule. The AI informs the human of a morning meeting with the product team, time to work on the LangChain project, and a lunch meeting with a customer interested in the latest AI developments. The human asks what would be a good demo to show.
AI: Based on the customer's interest in AI developments, I would suggest showcasing our latest natural language processing capabilities. We could demonstrate how our AI can accurately understand and respond to complex language queries, and even provide personalized recommendations based on the user's preferences. Additionally, we could highlight our AI's ability to learn and adapt over time, making it a valuable tool for businesses looking to improve their customer experience.


[None, None]